In [1]:
import urllib.request
import pickle
from os import path
import numpy as np

In [2]:
PK_PATH = '/Users/fulop/Downloads/imdb.pkl'

In [3]:
if not path.isfile(PK_PATH):
    urllib.request.urlretrieve('https://s3.amazonaws.com/text-datasets/imdb_full.pkl', PK_PATH)

In [4]:
f = open(PK_PATH, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [5]:
len(x_train)

25000

In [6]:
import tensorflow as tf

In [7]:
idx2word = tf.contrib.keras.datasets.imdb.get_word_index()

In [8]:
idx2word = {v: k for k, v in idx2word.items()}

In [9]:
idx2word[23022]

'bromwell'

In [10]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [11]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [12]:
trn = tf.contrib.keras.preprocessing.sequence.pad_sequences(trn, maxlen=500, value=0)
test = tf.contrib.keras.preprocessing.sequence.pad_sequences(test, maxlen=500, value=0)

In [13]:
trn.shape

(25000, 500)

In [14]:
tf.reset_default_graph()
x_inputs = tf.placeholder(tf.int32, shape=[None, 500])
y_labels = tf.placeholder(tf.float32, shape=[None, 1])
embeddings = tf.Variable(
    tf.random_uniform([5000, 32], -1.0, 1.0))
embeddings = tf.nn.embedding_lookup(embeddings, x_inputs)
embeddings = tf.reshape(embeddings, [-1,16000])
embeddings = tf.layers.dense(
                    embeddings,
                    100,
                    activation=tf.nn.relu,
                    name='dense1',
                )
embeddings = tf.layers.dropout(embeddings, 0.7, name="dense_dropout")
embeddings = tf.layers.dense(
                    embeddings,
                    1,
                    activation=tf.nn.sigmoid,
                    name='dense2',
                )


In [15]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=embeddings, labels=y_labels))

In [16]:
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

In [17]:
sess = tf.Session()

In [18]:
sess.run(tf.global_variables_initializer())


In [19]:
print(np.reshape(labels_train,[-1,1]).shape)

(25000, 1)


In [20]:
labels_train = np.reshape(labels_train, [-1, 1])
feed_dict = {x_inputs: trn, y_labels: labels_train}
res = sess.run([optimizer, loss], feed_dict=feed_dict)

In [21]:
print(res)

[None, 0.75212365]


In [22]:
labels_train = np.reshape(labels_train, [-1, 1])
feed_dict = {x_inputs: trn, y_labels: labels_train}
res = sess.run([optimizer, loss], feed_dict=feed_dict)

In [23]:
print(res)

[None, 0.69310421]


In [24]:
sess.close()

In [27]:
tf.reset_default_graph()
x_inputs = tf.placeholder(tf.int32, shape=[None, 500])
y_labels = tf.placeholder(tf.float32, shape=[None, 1])
embeddings = tf.Variable(
    tf.random_uniform([5000, 32], -1.0, 1.0))
embeddings = tf.nn.embedding_lookup(embeddings, x_inputs)
embeddings = tf.layers.dropout(embeddings, 0.2)
embeddings = tf.layers.conv1d(
      inputs=embeddings,
      filters=64,
      kernel_size=5,
      padding="same",
      activation=tf.nn.relu)
embeddings = tf.layers.dropout(embeddings, 0.2)
embeddings = tf.layers.max_pooling1d(embeddings, pool_size=2,
                                       strides=2)
embeddings = tf.reshape(embeddings, [-1,16000])
embeddings = tf.layers.dense(
                    embeddings,
                    100,
                    activation=tf.nn.relu,
                    name='dense1',
                )
embeddings = tf.layers.dropout(embeddings, 0.7, name="dense_dropout")
embeddings = tf.layers.dense(
                    embeddings,
                    1,
                    activation=tf.nn.sigmoid,
                    name='dense2',
                )
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=embeddings, labels=y_labels))
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

labels_train = np.reshape(labels_train, [-1, 1])
feed_dict = {x_inputs: trn, y_labels: labels_train}
res = sess.run([optimizer, loss], feed_dict=feed_dict)
res = sess.run([optimizer, loss], feed_dict=feed_dict)
print(res)

[None, 0.69313937]


In [28]:
sess.close()

In [30]:
GLOVE_PATH = '/Users/fulop/Downloads/6B.50d.tgz'

In [31]:
if not path.isfile(GLOVE_PATH):
    urllib.request.urlretrieve('http://files.fast.ai/models/glove/6B.50d.tgz', GLOVE_PATH)

In [32]:
GLOVE_FOLDER = '/Users/fulop/Downloads/6B.50d/6B.50d'

In [39]:
import bcolz

In [46]:
open(GLOVE_FOLDER+'_words.pkl', 'rb')

<_io.BufferedReader name='/Users/fulop/Downloads/6B.50d/6B.50d_words.pkl'>

In [51]:
def load_vectors():
    return (bcolz.open(GLOVE_FOLDER+'.dat')[:],
        pickle.load(open(GLOVE_FOLDER+'_words.pkl','rb'), encoding='utf-8'),
        pickle.load(open(GLOVE_FOLDER+'_idx.pkl','rb'), encoding='utf-8'))

In [52]:
vecs, words, wordidx = load_vectors()

In [60]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = np.random.normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = np.random.normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [61]:
import re

In [64]:
print(emb[10])

[  3.96366666e-02   5.08499990e-02  -2.73576677e-02  -2.47146666e-01
   2.53056665e-01  -1.61093334e-01  -1.03363335e-01   1.71586672e-01
  -3.29026659e-01   2.05856670e-04  -5.01433313e-02   2.79233336e-01
  -3.59899998e-01  -1.71533326e-01   4.39599991e-01   2.06689994e-01
   4.59299982e-02   1.57026668e-01  -2.42913341e-02  -2.42250005e-01
  -2.47053325e-01   2.50876665e-01   2.93933332e-01   9.85366702e-02
   4.51599995e-01  -8.56700023e-01  -4.50766683e-01   1.52933329e-01
   3.35600019e-01  -3.95200014e-01   1.15790002e+00   2.59660006e-01
  -2.43096670e-01   8.36733381e-02  -8.71866643e-02  -1.15613331e-01
   1.86136663e-01   2.50326673e-01   1.66099995e-01  -8.94099971e-02
  -9.14766686e-04  -6.09933337e-03  -9.36533312e-02   1.84393326e-01
   1.25686663e-02   6.18500014e-02  -5.00833342e-02  -1.91706657e-01
  -8.89033377e-02   3.07069997e-01]


In [67]:
print(type(emb))

<class 'numpy.ndarray'>


In [69]:
emb = emb.astype(float)

In [71]:
tf.reset_default_graph()
x_inputs = tf.placeholder(tf.int32, shape=[None, 500])
y_labels = tf.placeholder(tf.float32, shape=[None, 1])
embeddings = tf.Variable(emb)
embeddings = tf.cast(embeddings, tf.float32)
embeddings = tf.nn.embedding_lookup(embeddings, x_inputs)
embeddings = tf.layers.dropout(embeddings, 0.2)
embeddings = tf.layers.conv1d(
      inputs=embeddings,
      filters=64,
      kernel_size=5,
      padding="same",
      activation=tf.nn.relu)
embeddings = tf.layers.dropout(embeddings, 0.2)
embeddings = tf.layers.max_pooling1d(embeddings, pool_size=2,
                                       strides=2)
embeddings = tf.reshape(embeddings, [-1,16000])
embeddings = tf.layers.dense(
                    embeddings,
                    100,
                    activation=tf.nn.relu,
                    name='dense1',
                )
embeddings = tf.layers.dropout(embeddings, 0.7, name="dense_dropout")
embeddings = tf.layers.dense(
                    embeddings,
                    1,
                    activation=tf.nn.sigmoid,
                    name='dense2',
                )
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=embeddings, labels=y_labels))
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

labels_train = np.reshape(labels_train, [-1, 1])
feed_dict = {x_inputs: trn, y_labels: labels_train}
res = sess.run([optimizer, loss], feed_dict=feed_dict)
res = sess.run([optimizer, loss], feed_dict=feed_dict)
print(res)

[None, 0.69313937]
